In [85]:
import pandas as pd
import joblib
import glob
import os 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
import plotly.graph_objects as go

In [86]:
def create_pred(BASE_PATH, model_name, iter, api= True):

    if api:
        DATA_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/data/test.csv")
        MODEL_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/models/")

        X = pd.read_csv(DATA_PATH)
    
        X_test = X.drop(columns=["Solar_MWh_credit"])
        y_test = X["Solar_MWh_credit"]

        model_paths = glob.glob(MODEL_PATH + f"/{model_name}/models/i{iter}_models/*.pkl")

        predictions = {model_path: joblib.load(model_path).predict(X_test) for model_path in model_paths}

        alpha_val = {"q1": 0.1, "q2": 0.2, "q3": 0.3, "q4": 0.4, "q5": 0.5, "q6": 0.6, "q7": 0.7, "q8": 0.8, "q9": 0.9}

        mean_pinball_losses = {model_path: mean_pinball_loss(y_test, predictions[model_path], alpha= alpha_val[model_path.split("_")[-1].split(".")[0]]) for model_path in model_paths}

    else: 
        DATA_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/data/train.csv")
        MODEL_PATH = os.path.join(BASE_PATH, "Generation_forecast/Solar_forecast/models/")

        df = pd.read_csv(DATA_PATH)
        X = df.drop(columns=["Solar_MWh_credit"])
        y = df["Solar_MWh_credit"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 0)

        model_paths = glob.glob(MODEL_PATH + f"/{model_name}/models/i{iter}_models/*.pkl")

        predictions = {model_path: joblib.load(model_path).predict(X_test) for model_path in model_paths}

        alpha_val = {"q1": 0.1, "q2": 0.2, "q3": 0.3, "q4": 0.4, "q5": 0.5, "q6": 0.6, "q7": 0.7, "q8": 0.8, "q9": 0.9}

        mean_pinball_losses = {model_path: mean_pinball_loss(y_test, predictions[model_path], alpha= alpha_val[model_path.split("_")[-1].split(".")[0]]) for model_path in model_paths}


    return y_test, predictions, mean_pinball_losses, model_paths

In [87]:
def plot_results(y_test, predictions, mean_pinball_losses, model_name, iter):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=y_test, name="True Values", mode="lines", line= dict(color= "black")))
    for model_path, prediction in predictions.items():
        fig.add_trace(go.Scatter(y=prediction, name=f"Predictions {model_path.split('/')[-1]}, Loss: {mean_pinball_losses[model_path]}", mode="lines"))
    
    fig.update_layout(title=f"True Values vs Predictions of {model_name}, iteration {iter}", xaxis_title="Time", yaxis_title="Solar Production (MWh)")
    fig.show()

### Eval Models

In [88]:
# Available models and current iterations -- DO NOT CHANGE !!!
models = {
    "hgbr": {"name": "hgbr_model", "iteration": 2},
    #"xgbr": {"name": "xgbr_model", "iteration": 1}, Currently not working need to train new iteration with correct parameters
    "lgbr": {"name": "lgbr_model", "iteration": 4}
}

# Set BASE_PATH to the root directory of the project and choose the model to evaluate
BASE_PATH = os.getenv('BASE_PATH', "/Users/florian/Documents/github/DP2/Energy_production_price_prediction/") 
eval_model_key = "lgbr"


# Create the predictions and plot the results -- DO NOT CHANGE !!!
eval_model_name = models[eval_model_key]["name"]
eval_iter = models[eval_model_key]["iteration"]
y_test, predictions, mean_pinball_losses, model_paths = create_pred(BASE_PATH= BASE_PATH, model_name= eval_model_name, iter= eval_iter, api= True)
plot_results(y_test, predictions, mean_pinball_losses, model_name= eval_model_name, iter= eval_iter)

In [89]:
for model_path in model_paths:
    error = np.mean(np.abs(y_test - predictions[model_path]))
    print(model_path.split('/')[-1], error)

lgbr_q8.pkl 19.169806685895484
lgbr_q9.pkl 22.33084899304047
lgbr_q1.pkl 50.57305639670764
lgbr_q2.pkl 29.260317640102986
lgbr_q3.pkl 32.8149032452733
lgbr_q7.pkl 22.72405050953386
lgbr_q6.pkl 28.92453595974705
lgbr_q4.pkl 32.04037691515679
lgbr_q5.pkl 28.478135121252848
